In [1]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

sys.path.insert(0, '../')
import sc_pull

%load_ext autoreload
%autoreload 1
pd.set_option('display.max_columns', 500)

In [2]:
with urllib.request.urlopen("https://api.hpc.tools/v1/public/fts/flow?globalClusterCode=shl") as url:
    data = json.loads(url.read().decode())

ret = json_normalize(data)

#total number of flows: 9,294
ret['data.incoming.flowCount']

#total amount of incoming funding: 7,857,348,149
ret['data.incoming.fundingTotal']

#pledged: 37,481,772
ret['data.incoming.fundingTotal']

0    7906537240
Name: data.incoming.fundingTotal, dtype: int64

In [3]:
import imp
imp.reload(sc_pull)

loaded 227 SC entries


<module 'sc_pull' from '..//sc_pull.py'>

In [575]:
#outside
# j = j.merge(hist, left_on = 'fts.url', right_on = 'sc.fts_url')

In [258]:
class fts(object):

    def __init__(self, test = None, all_sector = False):
        self.all_sector = all_sector
        self.test = test
        
        self.fts = self.master_pull()
        self.merge_sc()
        self.flows = self.extract_flows()
        
        #remove flows col as it's quite large
        self.fts = self.fts.drop('fts.data.flows', axis = 1)
        
    def extract_flows(self):
        """take all flows and add to df"""
        
        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.data.flows'])
            norm['sc.uid'] = v[1]['sc.uid']

            if len(flow_df) == 0:
                flow_df = norm
            else:
                #need to make sure we're adding t3h same c0lz
                assert(norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index = True)
        
        return flow_df
    
    def master_pull(self):
        """group by year, country and pull down total values with optional filter by cluster"""
        countries = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode()))['data']

        print('pulled countries')

        self.hist = sc_pull.pull()

        self.hist['sc.fts_url'] = self.hist.apply(lambda x : \
                        'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1}{2}' \
                        .format(x['sc.iso3'], x['sc.year'], \
                        '&globalClusterCode=shl' if not self.all_sector else ''), axis = 1)
        
        if self.test:
            hrefz = self.hist['sc.fts_url'][:self.test]
        else:
            hrefz = self.hist['sc.fts_url']

        def exception_handler(request, exception):
             print('Bad URL for ' + request)

        resps = []
        rs = (grequests.get(ref) for ref in hrefz)
        resps += grequests.map(rs, exception_handler = exception_handler, size = 25)

        good_resps = []
        bad_resps = []
        for r in resps:
            load = json.loads(r.content)
            load['url'] = r.url
            if r.status_code == 200:
                good_resps.append(load)
            else:
                bad_resps.append(load)
        
        print('num bad resps: ' + str(len(bad_resps)))
        return json_normalize(good_resps).add_prefix('fts.')
    
    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on = 'fts.url', right_on = 'sc.fts_url')
    
    def get_flow_bdown(self):
        def hp(v):
            if 'Plan' in json_normalize(v['destinationObjects']).values or 'Plan' in json_normalize(v['sourceObjects']).values:
                return 'has_plan'
            else:
                return 'no_plan'

        self.flows['plan'] = self.flows.apply(lambda x : hp(x), axis = 1)

        #we are skipping boundary for now as it's <<<< 
        d = self.flows[['sc.uid', 'plan', 'amountUSD']].groupby(['sc.uid', 'plan'], as_index = False).aggregate({'amountUSD' : 'sum'})
        piv = pd.pivot_table(d, index = 'sc.uid', values='amountUSD', aggfunc=np.sum, columns = [d['plan']] \
                             , fill_value = 0).reset_index()
        piv['total'] = piv['has_plan'] + piv['no_plan']
        return piv

In [290]:
all_f = fts(all_sector = True)
s_f = fts()

all_b = all_f.get_flow_bdown()
all_b.rename({'total' : 'all_total'}, inplace = True, axis = 1)

sf_b = s_f.get_flow_bdown()
sf_b.rename({'total' : 'sc_total'}, inplace = True, axis = 1)

mrgd = sf_b.merge(all_b[['sc.uid', 'all_total']], how = 'left', on = 'sc.uid')
mrgd['all_total'] = mrgd['all_total'] - mrgd['sc_total']

pulled countries
loaded 227 SC entries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


pulled countries
loaded 227 SC entries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/ipykernel_launcher.py:72: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [291]:
mrgd

plan,sc.uid,has_plan,no_plan,sc_total,all_total
0,AFG2008,0,5591549,5591549,545858739
1,AFG2009,25472179,3513009,28985188,607221589
2,AFG2010,31890043,3900869,35790912,541932989
3,AFG2011,8163742,9171957,17335699,597155660
4,AFG2012,11385904,1195950,12581854,451615156
5,AFG2013,32521337,149591,32670928,301588421
6,AFG2014,19520480,944681,20465161,502716700
7,AFG2015,5470535,1936544,7407079,494685402
8,AFG2016,14039028,408879,14447907,354633642
9,AFG2017,28094066,4127689,32221755,323034522


In [580]:
g = pd.DataFrame(pd.pivot_table(r.flows, index='sc.uid', columns=['flow.'+r.flows['boundary']+'_'+r.flows['status']], values='amountUSD', aggfunc=np.sum)).merge(r.fts, how='left', on='sc.uid')
g

,sc.uid,incoming_commitment,incoming_paid,incoming_pledge,internal_commitment,internal_paid,outgoing_commitment,outgoing_paid,fts.data.incoming.flowCount,fts.data.incoming.fundingTotal,fts.data.incoming.pledgeTotal,fts.data.internal.flowCount,fts.data.outgoing.flowCount,fts.data.outgoing.fundingTotal,fts.data.outgoing.pledgeTotal,fts.meta.count,fts.meta.language,fts.status,fts.url,sc.region,sc.region_url,sc.country,sc.country_url,sc.year,sc.response,sc.lead_agency,sc.disaster_type,sc.website_url,sc.colead_agency,sc.activation_date,sc.deactivation_date,sc.num_clust,sc.iso3,sc.fts_url
0,AFG2008,5395287.0,196262.0,NaN,NaN,NaN,NaN,NaN,9,5591549,0,0,0,0,0,9,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2008,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,IOM,02.03.2008,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
1,AFG2009,10572700.0,18412488.0,NaN,NaN,NaN,NaN,NaN,27,28985188,0,0,0,0,0,27,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2009,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
2,AFG2010,10625080.0,25165832.0,NaN,NaN,NaN,NaN,NaN,30,35790912,0,0,0,0,0,30,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2010,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
3,AFG2011,5797689.0,11538010.0,NaN,NaN,NaN,NaN,NaN,21,17335699,0,0,0,0,0,21,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2011,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
4,AFG2012,3221427.0,8775546.0,NaN,NaN,NaN,584881.0,NaN,13,11996973,0,0,1,584881,0,14,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2012,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
5,AFG2013,10706956.0,21639972.0,324000.0,NaN,NaN,NaN,NaN,28,32346928,324000,0,0,0,0,28,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2013,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
6,AFG2014,10832372.0,9632789.0,NaN,NaN,NaN,NaN,NaN,24,20465161,0,0,0,0,0,24,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2014,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
7,AFG2015,1365912.0,6041167.0,NaN,NaN,NaN,NaN,NaN,20,7407079,0,0,0,0,0,20,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://sheltercluster.org/asiapacific,Afghanistan,http://sheltercluster.org/asiapacific/afghanistan,2015,Afghanistan - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/afghanistan,NaN,NaN,NaN,1,AFG,https://api.hpc.tools/v1/public/fts/flow?count...
8,AFG2016,2718091.0,11729816.0,NaN,NaN,NaN,NaN,NaN,21,14447907,0,0,0,0,0,21,en,ok,https://api.hpc.tools/v1/public/fts/flow?count...,Asia-Pacific,http://shelterclust

In [581]:
gran_cols = ['incoming_commitment', 'incoming_paid', 'incoming_pledge', 'outgoing_commitment', 
            'outgoing_paid', 'outgoing_pledge', 'internal_commitment', 'internal_paid', 'internal_pledge']


def check_sum(uid):
    """check sums are good"""
    sm = sum([ft[ft['sc.uid'] == uid][g] for g in gran_cols if ft[ft['sc.uid'] == uid][g]][0])

def pop_flows():
    g = fs.groupby(by = ['sc.uid', 'boundary', 'status']).agg({'amountUSD' : 'sum'})
    g = fs[[‘amountUSD’,‘boundary', 'status’]].groupby(‘boundary', 'status’).sum()
    g = g.reset_index()
        
    for c in gran_cols:
        ft[c] = None

    for v in g.iterrows():
        var_nm = v[1]['boundary'] + '_' + v[1]['status']
        
        if len(ft.loc[ft['sc.uid'] == v[1]['sc.uid']]) != 1:
            raise Exception('bad UID')
        
        if v[1]['sc.uid'] not in list(ft['sc.uid']):
            print(v[1]['sc.uid'])
        
        ft.loc[ft['sc.uid'] == v[1]['sc.uid'], var_nm] = v[1]['amountUSD']
        
        #check all sums are good
#         check_sum(v[1]['sc.uid'])
        
#     df[col_list].sum(axis=1)
    
pop_flows()

SyntaxError: invalid character in identifier (<ipython-input-581-41cc2d97e67c>, line 11)

In [ ]:
ft

In [ ]:
gran_cols = ['incoming_commitment', 'incoming_paid', 'incoming_pledge', 'outgoing_commitment', 
            'outgoing_paid', 'outgoing_pledge', 'internal_commitment', 'internal_paid', 'internal_pledge']

s = sum([ft[ft['sc.uid'] == 'PAK2005'][g] for g in gran_cols if ft[ft['sc.uid'] == 'PAK2005'][g][0]]) -1

In [ ]:
(ft[ft['sc.uid'] == 'PAK2005']['fts.data.incoming.fundingTotal'] + \
ft[ft['sc.uid'] == 'PAK2005']['fts.data.incoming.pledgeTotal']) == s

In [ ]:
ft

In [ ]:
g = fs.groupby(by = ['sc.uid', 'boundary', 'status']).agg({'amountUSD' : 'sum'})

g.reset_index()

ret = collections.OrderedDict({
        'inc_commitment' : None,
        'inc_paid' : None,
        'inc_pledge' : None,
        'out_commitment' : None,
        'out_paid' : None,
        'out_pledge' : None
        })

#add cols to fs
for v in ret.keys():
    ft[v] = None

In [ ]:
def cost_bdown(v):
    """breakdown cost by commitment, paid and pledge.
        create cols:
            inc_commitment
            inc_paid
            inc_pledge
            out_commitment
            out_paid
            out_pledge
    """
    ret = collections.OrderedDict({
            'inc_commitment' : None,
            'inc_paid' : None,
            'inc_pledge' : None,
            'out_commitment' : None,
            'out_paid' : None,
            'out_pledge' : None
            })
    
    #add cols to fs
    for v in ret.keys():
        fs[v] = None
    
    if len(v['fts.data.flows']) > 0:
        gpd = json_normalize(v['fts.data.flows'][0]).groupby(by = 'status')['amountUSD'].sum()
    
        for k in gpd.keys():
            if k in ret:
                ret[k] = gpd[k]
    
    print(pd.Series(ret))
    return ret
            

def p(v):
    print(v['fts.data.flows'])
        
# j.apply(lambda x : cost_bdown(x), axis = 1)
j[['amt_commit', 'amt_paid', 'amt_pledge']] = j.apply(lambda x: pd.Series(cost_bdown(x)), axis = 1)

In [ ]:
bounds = {}
def get_bound_types(x):
    """see what count boundary transactions we have
        {'incoming': 4889, 'internal': 10, 'outgoing': 23}
    """
    norm = json_normalize(x['fts.data.flows'])
    
    if 'boundary' in norm:
        for v in norm['boundary']:
            if v == 'outgoing':
                print(x)
                raise('sfd')

            if v not in bounds:
                bounds[v] = 1
            else:
                bounds[v] += 1
    
j.apply(lambda x : get_bound_types(x), axis = 1)

In [ ]:
#test to see if we're figuring out outgoing correctly
ent = j[j['fts.data.incoming.fundingTotal'] == 1597544]

#regular:
json_normalize(ent['fts.data.flows'][9]).groupby(by = 'status')['amountUSD'].sum()
# commitment    1130923
# paid           616621

#subtract:
r = json_normalize(ent['fts.data.flows'][9]).groupby(by = ['status', 'boundary'])['amountUSD'].sum()

list(set(r.index.get_level_values('status')))

r.keys()